In [1]:
import pandas as pd
import yfinance as yf
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# 데이터 불러오기
start = datetime.datetime(2020, 1, 1)
end = datetime.datetime(2024, 12, 27)
btc_data = yf.download('BTC-USD', start=start, end=end)

# 필요한 컬럼 선택
btc_data = btc_data[['Close']]

[*********************100%***********************]  1 of 1 completed


In [2]:
# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(btc_data)

# 학습 데이터 생성 함수
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

In [3]:
# 데이터 차원 변환 [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

In [4]:
# 모델 생성 (최적 파라미터 적용)
model = Sequential([
    LSTM(200, return_sequences=True, input_shape=(time_step, 1)),
    Dropout(0.2),
    LSTM(200, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

# 모델 컴파일
model.compile(optimizer=Adam(), loss='mean_squared_error')

# 모델 학습
model.fit(X, y, epochs=50, batch_size=32, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.0226
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0011
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0012
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0010
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0010
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0012
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0011
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 9.0811e-04
Epoch 9/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0011
Epoch 10/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 9.1986e-04
Epoch 11/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 7.9460e-04
Epoch 12/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 8.6610e-04
Epoch 13/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 9.3093e-04
Epoch 14/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 7.5678e-04
Epoch 15/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/

In [5]:
# 예측 데이터 생성
train_size = int(len(scaled_data) * 0.8)
test_data = scaled_data[train_size - time_step:]

X_test, y_test = create_dataset(test_data, time_step)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 예측
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# 예측 결과 저장
results_df = pd.DataFrame({'datetime': btc_data.index[-len(predictions):], 'prediction': predictions.flatten()})
results_df.to_csv("lstm_predictions.csv", index=False)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step


In [46]:
# 실제 Close 가격과 예측값을 함께 저장하기
btc_data = btc_data[['Close']]
df = pd.read_csv("lstm_predictions.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# 예측값과 실제 Close 가격을 합친 데이터프레임 만들기
btc_data = btc_data.loc[df.index]  # 예측값과 일치하는 날짜의 실제 Close 가격
df['actual_close'] = btc_data['Close']

# DataFrame을 다시 저장.
df.to_csv("lstm_predictions_with_actual.csv", index=True)

In [7]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 12.9 MB/s eta 0:00:00


In [57]:
#LSTM 예측값을 Backtrader에서 사용할 수 있도록
import backtrader as bt
import pandas as pd

# LSTM 예측값 불러오기
df = pd.read_csv("lstm_predictions_with_actual.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Backtrader용 데이터 피드 생성
class LSTMPredictFeed(bt.feeds.PandasData):
    lines = ('prediction', 'actual_close')  # 예측값과 실제 가격 추가
    params = (
        ('prediction', -1),  # 예측값 컬럼
        ('actual_close', -1)  # 실제 가격 컬럼
    )

In [58]:
class LSTMPredictStrategy(bt.Strategy):
    params = dict(threshold=0.002)  # 예측값과 현재 가격 차이 기준

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt}: {txt}")  # 로그 출력

    def next(self):
        current_price = self.data.actual_close[0]
        predicted_price = self.data.prediction[0]

        # 매매 전략 로그 출력
        self.log(f"현재 가격: {current_price}, 예측 가격: {predicted_price}")

        if predicted_price > current_price * (1 + self.p.threshold):  # 예측값이 현재보다 높으면 매수
            if not self.position:
                self.log("매수 주문 실행")
                self.buy()
        elif predicted_price < current_price * (1 - self.p.threshold):  # 예측값이 현재보다 낮으면 매도
            if self.position:
                self.log("매도 주문 실행")
                self.sell()
        # 포트폴리오 값 로그 추가
        print(f"Portfolio Value: {self.broker.getvalue()}")


In [59]:
#Backtrader 백테스팅 실행
# Cerebro 엔진 생성
cerebro = bt.Cerebro()
cerebro.broker.set_cash(10000)  # 초기 자본금

# 데이터 로드
data = LSTMPredictFeed(dataname=df)
cerebro.adddata(data)

# 전략 추가
cerebro.addstrategy(LSTMPredictStrategy)

# 초기 투자금 설정
cerebro.broker.set_cash(10000)

# 수수료 설정 (예: 0.1%)
cerebro.broker.setcommission(commission=0.001)

# 백테스트 실행
print("Starting Portfolio Value:", cerebro.broker.getvalue())
cerebro.run()
print("Final Portfolio Value:", cerebro.broker.getvalue())

# 결과 시각화
cerebro.plot()

Starting Portfolio Value: 10000
2023-12-29: 현재 가격: 42099.40234375, 예측 가격: 41976.04
Portfolio Value: 10000.0
2023-12-30: 현재 가격: 42156.90234375, 예측 가격: 41794.03
Portfolio Value: nan
2023-12-31: 현재 가격: 42265.1875, 예측 가격: 41371.062
Portfolio Value: nan
2024-01-01: 현재 가격: 44167.33203125, 예측 가격: 41128.86
Portfolio Value: nan
2024-01-02: 현재 가격: 44957.96875, 예측 가격: 41109.543
Portfolio Value: nan
2024-01-03: 현재 가격: 42848.17578125, 예측 가격: 42103.473
Portfolio Value: nan
2024-01-04: 현재 가격: 44179.921875, 예측 가격: 43245.758
Portfolio Value: nan
2024-01-05: 현재 가격: 44162.69140625, 예측 가격: 42794.02
Portfolio Value: nan
2024-01-06: 현재 가격: 43989.1953125, 예측 가격: 42822.52
Portfolio Value: nan
2024-01-07: 현재 가격: 43943.09765625, 예측 가격: 42929.18
Portfolio Value: nan
2024-01-08: 현재 가격: 46970.50390625, 예측 가격: 42919.746
Portfolio Value: nan
2024-01-09: 현재 가격: 46139.73046875, 예측 가격: 42865.133
Portfolio Value: nan
2024-01-10: 현재 가격: 46627.77734375, 예측 가격: 44336.863
Portfolio Value: nan
2024-01-11: 현재 가격: 46368.585937

ValueError: Axis limits cannot be NaN or Inf

In [50]:
results = cerebro.run()
print("Results:", results)

2023-12-29: 현재 가격: 42099.40234375, 예측 가격: 41976.04
Portfolio Value: 10000.0
2023-12-30: 현재 가격: 42156.90234375, 예측 가격: 41794.03
Portfolio Value: nan
2023-12-31: 현재 가격: 42265.1875, 예측 가격: 41371.062
Portfolio Value: nan
2024-01-01: 현재 가격: 44167.33203125, 예측 가격: 41128.86
Portfolio Value: nan
2024-01-02: 현재 가격: 44957.96875, 예측 가격: 41109.543
Portfolio Value: nan
2024-01-03: 현재 가격: 42848.17578125, 예측 가격: 42103.473
Portfolio Value: nan
2024-01-04: 현재 가격: 44179.921875, 예측 가격: 43245.758
Portfolio Value: nan
2024-01-05: 현재 가격: 44162.69140625, 예측 가격: 42794.02
Portfolio Value: nan
2024-01-06: 현재 가격: 43989.1953125, 예측 가격: 42822.52
Portfolio Value: nan
2024-01-07: 현재 가격: 43943.09765625, 예측 가격: 42929.18
Portfolio Value: nan
2024-01-08: 현재 가격: 46970.50390625, 예측 가격: 42919.746
Portfolio Value: nan
2024-01-09: 현재 가격: 46139.73046875, 예측 가격: 42865.133
Portfolio Value: nan
2024-01-10: 현재 가격: 46627.77734375, 예측 가격: 44336.863
Portfolio Value: nan
2024-01-11: 현재 가격: 46368.5859375, 예측 가격: 45023.945
Portfolio Va

In [52]:
df = pd.read_csv("lstm_predictions_with_actual.csv")
print(df.head())
print(df.info())

     datetime  prediction  actual_close
0  2023-12-29   41976.040  42099.402344
1  2023-12-30   41794.030  42156.902344
2  2023-12-31   41371.062  42265.187500
3  2024-01-01   41128.860  44167.332031
4  2024-01-02   41109.543  44957.968750
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datetime      364 non-null    object 
 1   prediction    364 non-null    float64
 2   actual_close  364 non-null    float64
dtypes: float64(2), object(1)
memory usage: 8.7+ KB
None
